In [35]:
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch
import numpy as np
import pandas as pd
#import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import re
import warnings
import math
warnings.simplefilter(action='ignore', category=FutureWarning)

In [36]:
dataset = pd.read_csv("TraningSet(postLabel_1Label_2).csv", sep=',',engine='python')
#dataset = pd.read_csv("Traningset-TFND_toxicity_features_new(label1_label2).csv", sep='\t',engine='python')
#dataset.Stance = pd.Categorical(pd.factorize(dataset.Stance)[0])

In [37]:
dataset.head()

,news,label_1,label_2
0,The U.S. CDC quietly updated their numbers in ...,1,False
1,Social media posts shared thousands of times a...,0,False
2,The CDC updated the COVID-19 number to admit t...,0,False
3,Either masks work or they donâ€™t; if masks wo...,1,False
4,The RT-PCR test for the virus that causes COVI...,1,False


In [38]:
test_dataset = pd.read_csv("TestSet(postLabel_1Label_2).csv", sep=',',engine='python')
#test_dataset.Stance = pd.Categorical(pd.factorize(test_dataset.Stance)[0])

In [39]:
'''train_text = []
for i in range(len(dataset['Tweet'])):
    train_text.append(dataset['Target'][i] + " "+ dataset['Tweet'][i])
'''

'train_text = []\nfor i in range(len(dataset[\'Tweet\'])):\n    train_text.append(dataset[\'Target\'][i] + " "+ dataset[\'Tweet\'][i])\n'

In [40]:
'''test_text = []
for i in range(len(test_dataset['Tweet'])):
    test_text.append(test_dataset['Target'][i] + " "+ test_dataset['Tweet'][i])
    '''

'test_text = []\nfor i in range(len(test_dataset[\'Tweet\'])):\n    test_text.append(test_dataset[\'Target\'][i] + " "+ test_dataset[\'Tweet\'][i])\n    '

In [41]:
PRE_TRAINED_MODEL_NAME = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)
bert_model = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [42]:
def extract_bert_features(dataset,filename):
    bertFile = open(filename,"w")
    for i in range(len(dataset)):
        encoding = tokenizer.encode_plus(
          dataset[i],
          max_length=38,
          add_special_tokens=True, # Add '[CLS]' and '[SEP]'
          return_token_type_ids=False,
          pad_to_max_length=True,
          return_attention_mask=True,
          truncation = True,
          return_tensors='pt',  # Return PyTorch tensors
        )
        #print(encoding)
        last_hidden_state, pooled_output = bert_model(
          input_ids=encoding['input_ids'],
          attention_mask=encoding['attention_mask']
        )
        bert_rep = (pooled_output.flatten().detach().numpy())
        #print(len(bert_rep))
        #print(bert_rep)
        for _ in range(len(bert_rep)-1):
            bertFile.write(repr(bert_rep[_])+",")
        bertFile.write(repr(bert_rep[767])+"\n")
    bertFile.close()

In [43]:
extract_bert_features(dataset['news'],"bert_representation.csv")

In [44]:
import pandas as pd
bert_dataset = pd.read_csv("bert_representation.csv", sep=',',engine='python',header=None)

In [45]:
extract_bert_features(test_dataset['news'],"test_bert_representation.csv")

In [46]:
import nltk
#nltk.download('punkt')
target_words = set()
for i in range(len(dataset['news'])):
    nltk_tokens = nltk.word_tokenize(dataset['news'][i].lower())
    for tok in nltk_tokens:
        target_words.add(tok)

target_words.remove("a")
target_words.remove("is")
target_words.remove("of")

In [47]:
target2id = {w:i for i,w in enumerate(target_words)}
def create_target_vector(dataset):
    target_dataset = []
    for target in dataset:
        target_vector = np.zeros(len(target2id))
        nltk_tokens = nltk.word_tokenize(target.lower())
        for tok in nltk_tokens:
            if tok in target2id:
                target_vector[target2id[tok]]=1
        target_dataset.append(target_vector)
    return(target_dataset)

In [48]:
train_target_vectors = (create_target_vector(dataset['news']))
test_target_vectors = (create_target_vector(test_dataset['news']))

In [49]:
bert_dataset = bert_dataset.to_numpy()


In [50]:
print(len(bert_dataset))

724


In [51]:
bert_target_train_ds  = []
for i in range(len(dataset['label_1'])):
    
    bert_target_vector = (np.concatenate((np.array(train_target_vectors[i]),(bert_dataset[i])),axis=None))
    bert_target_train_ds.append(bert_target_vector)
bert_target_train_ds[0]

array([ 0.        ,  0.        ,  0.        , ..., -0.74333525,
       -0.66287977,  0.55816853])

In [52]:
bert_target_test_ds  = []
test_bert_dataset = pd.read_csv("test_bert_representation.csv", sep=',',engine='python',header=None)
test_bert_dataset = test_bert_dataset.to_numpy()
for i in range(len(test_dataset['label_1'])):   
    bert_target_vector = (np.concatenate((np.array(test_target_vectors[i]),np.array(test_bert_dataset[i])),axis=None))
    bert_target_test_ds.append(bert_target_vector)
bert_target_test_ds[0][0:5]

array([0., 0., 0., 0., 0.])

In [53]:
from sklearn import svm
clf = svm.SVC(kernel='linear')
clf.fit(bert_target_train_ds, dataset['label_1'])

SVC(kernel='linear')

In [54]:
svm_test_pred = clf.predict(bert_target_test_ds)

In [55]:
svm_test_pred = clf.predict(bert_target_test_ds)
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(test_dataset['label_1'],svm_test_pred))
print(classification_report(test_dataset['label_1'],svm_test_pred,digits=4))

[[103  24]
 [ 49  21]]
              precision    recall  f1-score   support

           0     0.6776    0.8110    0.7384       127
           1     0.4667    0.3000    0.3652        70

    accuracy                         0.6294       197
   macro avg     0.5721    0.5555    0.5518       197
weighted avg     0.6027    0.6294    0.6058       197



In [71]:
svm_train_pred = clf.predict(bert_target_train_ds)
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(dataset['label_1'],svm_train_pred))
print(classification_report(dataset['label_1'],svm_train_pred,digits=4))

[[521   2]
 [  1 200]]
              precision    recall  f1-score   support

           0     0.9981    0.9962    0.9971       523
           1     0.9901    0.9950    0.9926       201

    accuracy                         0.9959       724
   macro avg     0.9941    0.9956    0.9948       724
weighted avg     0.9959    0.9959    0.9959       724



In [72]:
params_grid = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

In [73]:
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.svm import SVC
clf = GridSearchCV(SVC(), params_grid, cv=5)
clf.fit(bert_dataset, dataset['label_1'])

GridSearchCV(cv=5, estimator=SVC(),
             param_grid=[{'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001],
                          'kernel': ['rbf']},
                         {'C': [1, 10, 100, 1000], 'kernel': ['linear']}])

In [74]:
pred = clf.predict(test_bert_dataset)

In [81]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import fbeta_score
from sklearn.metrics import f1_score


print(classification_report(test_dataset['label_1'],pred))

print("f1_score:",f1_score(test_dataset['label_1'], pred.round(), average='micro'))
print("f2_score:",fbeta_score(test_dataset['label_1'],pred.round(), beta=2,average='micro'))
print("f0.5_score:",fbeta_score(test_dataset['label_1'], pred.round(), beta=0.5,average='micro'))


              precision    recall  f1-score   support

           0       0.65      1.00      0.79       127
           1       1.00      0.01      0.03        70

    accuracy                           0.65       197
   macro avg       0.82      0.51      0.41       197
weighted avg       0.77      0.65      0.52       197

f1_score: 0.649746192893401
f2_score: 0.649746192893401
f0.5_score: 0.649746192893401


In [60]:
from sklearn.ensemble import RandomForestClassifier

In [86]:
clf = RandomForestClassifier().fit(bert_target_train_ds, dataset['label_1'])

In [90]:
rf_pred = clf.predict(bert_target_test_ds)
print(classification_report(test_dataset['label_1'],rf_pred,digits=4))

print("f1_score:",f1_score(test_dataset['label_1'], rf_pred.round(), average='micro'))
print("f2_score:",fbeta_score(test_dataset['label_1'],rf_pred.round(), beta=2,average='micro'))
print("f0.5_score:",fbeta_score(test_dataset['label_1'], rf_pred.round(), beta=0.5,average='micro'))

              precision    recall  f1-score   support

           0     0.6513    1.0000    0.7888       127
           1     1.0000    0.0286    0.0556        70

    accuracy                         0.6548       197
   macro avg     0.8256    0.5143    0.4222       197
weighted avg     0.7752    0.6548    0.5283       197

f1_score: 0.6548223350253807
f2_score: 0.6548223350253807
f0.5_score: 0.6548223350253807


In [63]:
print("*************************************************************************************************")

*************************************************************************************************


In [64]:
print(svm_pred[0])
print(rf_pred[0])

NameError: name 'svm_pred' is not defined

In [ ]:
'''
nn_pred = pd.read_csv("bert-NN.csv", engine='python',header=None)
cnn_pred = pd.read_csv("bert-CNN.csv", engine='python',header=None)
'''

In [ ]:
'''
nn_pred[0][0]
cnn_pred[0][0]

'''

In [ ]:
'''mv_pred = []
for i in range(len(rf_pred)):
    voting = [0,0,0]
    voting[svm_pred[i]] = 1+voting[svm_pred[i]]
    voting[rf_pred[i]] = 1+voting[rf_pred[i]]
    voting[nn_pred[0][i]] = 1+voting[nn_pred[0][i]]
    voting[cnn_pred[0][i]] = 1+voting[cnn_pred[0][i]]
       
    mv_pred.append(voting.index(max(voting)))
    
    '''

In [ ]:
''' 
print(classification_report(test_dataset['Stance'],mv_pred))
'''